In [1]:
import torch
from torch import nn
from torch.utils.data import DataLoader
from torchvision import datasets
from torchvision.transforms import ToTensor

In [8]:
# Download training dataset
training_data = datasets.FashionMNIST(
    root="data",
    train=True,
    download=True,
    transform=ToTensor()
)

# Download testing dataset
testing_data = datasets.FashionMNIST(
    root="data",
    train=True,
    download=True,
    transform=ToTensor()
)

In [10]:
batch_size = 64

# Create dataloader
training_dataloader = DataLoader(training_data, batch_size=batch_size)
testing_dataloader = DataLoader(testing_data, batch_size=batch_size)

for X,y in testing_dataloader:
    print(f"Shape of x[N, C, H, W]: {X.shape}")
    print(f"Shape of y: {y.shape} {y.dtype}")
    break

Shape of x[N, C , H, W]: torch.Size([64, 1, 28, 28])
Shape of y: torch.Size([64]) torch.int64


In [11]:
# Get device
device = "cuda" if torch.cuda.is_available() else "mps" if torch.backends.mps.is_available() else "cpu"
print(f"Using {device} device")

Using cuda device


In [12]:
# Define model
class NeuralNetwork(nn.Module):
    def __init__(self):
        super().__init__()
        self.flatten = nn.Flatten()
        self.linear_relu_stack = nn.Sequential(
            nn.Linear(28*28, 512),
            nn.ReLU(),
            nn.Linear(512, 512),
            nn.ReLU(),
            nn.Linear(512, 10)
        )
        
    def forward(self, x):
        x = self.flatten(x)
        logits = self.linear_relu_stack(x)
        return logits
    
model = NeuralNetwork().to(device)
print(model)

NeuralNetwork(
  (flatten): Flatten(start_dim=1, end_dim=-1)
  (linear_relu_stack): Sequential(
    (0): Linear(in_features=784, out_features=512, bias=True)
    (1): ReLU()
    (2): Linear(in_features=512, out_features=512, bias=True)
    (3): ReLU()
    (4): Linear(in_features=512, out_features=10, bias=True)
  )
)


In [13]:
# Define loss function
loss_fn = nn.CrossEntropyLoss()
# Define optimizer
optimizer = torch.optim.SGD(model.parameters(), lr=1e-3)

In [14]:
def train(dataloader, model, loss_fn, optimizer):
    size = len(dataloader.dataset)
    model.train()
    for batch, (x,y) in enumerate(dataloader):
        x,y = x.to(device), y.to(device)
        
        # Compute loss
        pred = model(x)
        loss = loss_fn(pred, y)
        
        # Back propagation
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        
        # Ouput message
        if batch % 100 == 0:
            loss, current = loss.item(), batch*len(x)
            print(f"loss: {loss:>7f} [{current:>5d}/{size:>5d}]")

In [15]:
def test(dataloader, model, loss_fn):
    size = len(dataloader.dataset)
    num_batches = len(dataloader)
    model.eval()
    
    test_loss, correct = 0, 0
    with torch.no_grad():
        for x,y in dataloader:
            x,y = x.to(device), y.to(device)
            
            # Compute loss
            pred = model(x)
            test_loss += loss_fn(pred, y).item()
            correct += (pred.argmax(1) == y).type(torch.float).sum().item()
            
    test_loss /= num_batches
    correct /= size
    print(f"Test Error: \n Accuracy: {(100*correct):>0.1f}%, Avg loss: {test_loss:>.8f} \n")

In [17]:
# Train and test
epochs = 5
for t in range(epochs):
    print(f"Epoch {t+1}\n---------------------------------")
    train(training_dataloader, model, loss_fn, optimizer)
    test(testing_dataloader, model, loss_fn)
print("Done!")

Epoch 1
---------------------------------
loss: 1.145024 [    0/60000]
loss: 1.160785 [ 6400/60000]
loss: 0.965738 [12800/60000]
loss: 1.112100 [19200/60000]
loss: 0.982712 [25600/60000]
loss: 1.006278 [32000/60000]
loss: 1.053305 [38400/60000]
loss: 0.982118 [44800/60000]
loss: 1.041550 [51200/60000]
loss: 0.962108 [57600/60000]
Test Error: 
 Accuracy: 66.9%, Avg loss: 0.96666756 

Epoch 2
---------------------------------
loss: 1.027422 [    0/60000]
loss: 1.063718 [ 6400/60000]
loss: 0.853186 [12800/60000]
loss: 1.020763 [19200/60000]
loss: 0.896525 [25600/60000]
loss: 0.915326 [32000/60000]
loss: 0.976660 [38400/60000]
loss: 0.909838 [44800/60000]
loss: 0.965255 [51200/60000]
loss: 0.896037 [57600/60000]
Test Error: 
 Accuracy: 68.3%, Avg loss: 0.89418667 

Epoch 3
---------------------------------
loss: 0.941662 [    0/60000]
loss: 0.996164 [ 6400/60000]
loss: 0.772338 [12800/60000]
loss: 0.955954 [19200/60000]
loss: 0.839047 [25600/60000]
loss: 0.849164 [32000/60000]
loss: 0.9225

In [18]:
# Saving models
torch.save(model.state_dict(), "model.pth")
print("Save Pytorch Model State to model.pth")

Save Pytorch Model State to model.pth


In [19]:
# Load model
model = NeuralNetwork()
model.load_state_dict(torch.load("model.pth"))

<All keys matched successfully>

In [20]:
# Make predictions
classes = [
    "T-shirt/Top",
    "Trouser",
    "Pullover",
    "Dress",
    "Coat",
    "Sandal",
    "Shirt",
    "Sneaker",
    "Bag",
    "Ankle boot"
]

model.eval()
x, y = testing_data[0][0], testing_data[0][1]
with torch.no_grad():
    pred = model(x)
    predicted, actual = classes[pred[0].argmax(0)], classes[y]
    print(f'Predicted: "{predicted}", Actual: "{actual}"')

Predicted: "Ankle boot", Actual: "Ankle boot"
